In [2]:
import fastf1
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import LineString
import numpy as np
import sympy
from shapely.affinity import translate  


In [3]:
session = fastf1.get_session(2023, 'Monza', 'R')
session.load(telemetry=True, laps=True, weather=False)

req         WARNING 	DEFAULT CACHE ENABLED! (147.03 MB) C:\Users\KIRKE\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Italian Grand Prix - Race [v3.3.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '44', '23', '4', '14', '77', '40', '81', '2', '24', '10', '18', '27', '20', '31', '22']


In [4]:
laps = session.laps
lap_55_1 = laps.pick_driver('55').pick_lap(10).get_telemetry()
f1_api_coords = list(zip(lap_55_1["Y"],lap_55_1["X"]))

In [5]:
lap_55_1

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z
2,2023-09-03 13:36:20.203,0 days 01:35:19.158000,,NaN,0 days 00:00:00,10995,308,8,100,False,0,interpolation,0.071280,0.000012,OnTrack,-1381,-784,1869
3,2023-09-03 13:36:20.254,0 days 01:35:19.209000,,NaN,0 days 00:00:00.051000,11014,309,8,100,False,0,pos,4.465899,0.000771,OnTrack,-1380,-751,1871
4,2023-09-03 13:36:20.421,0 days 01:35:19.376000,,NaN,0 days 00:00:00.218000,11052,310,8,100,False,0,car,18.895000,0.003264,OnTrack,-1367,-608,1871
5,2023-09-03 13:36:20.474,0 days 01:35:19.429000,,NaN,0 days 00:00:00.271000,11091,311,8,100,False,0,pos,23.486759,0.004057,OnTrack,-1364,-562,1872
6,2023-09-03 13:36:20.714,0 days 01:35:19.669000,,NaN,0 days 00:00:00.511000,11131,312,8,100,False,0,pos,44.351875,0.007662,OnTrack,-1346,-355,1873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,2023-09-03 13:37:46.262,0 days 01:36:45.217000,,NaN,0 days 00:01:26.059000,10923,306,8,100,False,0,car,5712.850000,0.986909,OnTrack,-1414,-1178,1869
663,2023-09-03 13:37:46.274,0 days 01:36:45.229000,,NaN,0 days 00:01:26.071000,10965,307,8,100,False,0,pos,5713.875190,0.987086,OnTrack,-1413,-1165,1870
664,2023-09-03 13:37:46.454,0 days 01:36:45.409000,,NaN,0 days 00:01:26.251000,11007,308,8,100,False,0,pos,5729.289932,0.989749,OnTrack,-1395,-932,1871
665,2023-09-03 13:37:46.702,0 days 01:36:45.657000,,NaN,0 days 00:01:26.499000,11050,309,8,100,False,0,car,5750.616667,0.993433,OnTrack,-1374,-683,1871


There are two different systems, EPSG and UTM, look at this post https://docs.qgis.org/2.18/en/docs/gentle_gis_introduction/coordinate_reference_systems.html \
First we use UTM to get the zone from this image: https://docs.qgis.org/2.18/en/_images/utm_zones.png \
Then we convert the UTM zone into EPSG. We convert to EPSG since thats what pandas like. There is probably another reason aswell, find out

In [6]:
from pyproj import CRS

# or dictionary
crs = CRS.from_dict({'proj': 'utm', 'zone': 32, 'south': False})

print(crs.to_authority())  # ('EPSG', '32736')

('EPSG', '32632')


In [7]:
import geopandas as gpd
import folium

# Load the GeoDataFrame
monza_track = gpd.read_file("bacinger f1-circuits master circuits/it-1922.geojson")
centroid = monza_track.geometry.centroid.iloc[0]

# Ensure the GeoDataFrame is in WGS84 (lat/lon)
#monza_track = monza_track.to_crs(epsg=4326)

# Convert to a projected CRS suitable for buffering (e.g., UTM)
# Note: Choose the correct UTM zone for your specific data. Here, we assume zone 32N.
# This is something we add manually 
monza_track_projected = monza_track.to_crs(epsg=32632)

# Process the geometry to add width by buffering (buffer distance in meters)
width_in_meters = 5  # Specify the width of the track. Adjust according to needs.
monza_track_buffered = monza_track_projected.copy(deep = True)
monza_track_buffered['geometry'] = monza_track_buffered.geometry.buffer(width_in_meters)

C:\Users\KIRKE\AppData\Local\Temp\ipykernel_816\798706167.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = monza_track.geometry.centroid.iloc[0]


In [8]:
centroid.y

45.623470812745815

In [9]:
# Create a folium map centered at the centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=15)

# Add the GeoDataFrame to the map
folium.GeoJson(monza_track_buffered).add_to(m)

# Display the map
#m.save("monza_track_map.html")
#m  # This line is optional and just to display the map inline, if your

In [10]:
min_lon, min_lat = min(list(monza_track.geometry.iloc[0].coords))
max_lon, max_lat =max(list(monza_track.geometry.iloc[0].coords))

In [11]:
def compute_scaling(matrix, number):
    a = matrix[2]
    b = matrix[-1]
    return number * a*0.79+ b+0.0021 #larger is more to the right

def new_compute_scaling(matrix, number):
    a = matrix[2]
    b = matrix[-1]   #Larger is more stretched up
    return number * a*1.44 + b-0.00164 #Smaller number means lower down
    
def idk(f1_api_coords: list[tuple], monza_track: gpd.GeoDataFrame):
    api_min_lat,api_min_lon  = min(f1_api_coords)
    api_max_lat,api_max_lon  = max(f1_api_coords)
    min_lon, min_lat = min(list(monza_track.geometry.iloc[0].coords))
    max_lon, max_lat =max(list(monza_track.geometry.iloc[0].coords))

    lat_matrix = sympy.Matrix(np.array([[api_max_lat,1,max_lat],[api_min_lat,1,min_lat]])).rref()[0]
    lon_matrix = sympy.Matrix(np.array([[api_max_lon,1,max_lon],[api_min_lon,1,min_lon]])).rref()[0]

    scaled_api = [(compute_scaling(lon_matrix,lon),new_compute_scaling(lat_matrix,lat)) 
                  for lat, lon in f1_api_coords]
    return scaled_api
    
scaled = idk(f1_api_coords, monza_track)
original_line_string = LineString(scaled)
gdf = gpd.GeoDataFrame(geometry=[original_line_string], crs="EPSG:4326")  
new_projected = gdf.to_crs(epsg=32632)
kat = folium.Map(location=[centroid.y, centroid.x], zoom_start=14, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr="Esri")
folium.GeoJson(new_projected).add_to(kat)
kat

In [12]:
# Function to determine UTM zone from longitude
from pyproj import Proj, Transformer

def determine_utm_zone(longitude):
    return int((longitude + 180) / 6) + 1

# Define WGS84 as our coordinate reference system
proj_wgs84 = Proj(proj='latlong', datum='WGS84')

# Define UTM based on the origin's longitude
origin_lon, origin_lat =9.286713930962872,45.623470812745815
utm_zone = determine_utm_zone(origin_lon)
is_northern = origin_lat >= 0
proj_utm = Proj(proj='utm', zone=utm_zone, datum='WGS84', hemisphere='north' if is_northern else 'south')

# Set up a transformer between WGS84 and UTM
transformer_to_utm = Transformer.from_proj(proj_wgs84, proj_utm)
transformer_to_wgs84 = Transformer.from_proj(proj_utm, proj_wgs84)

# Function to convert relative coordinates to lat/lon
def relative_to_latlon(origin, relative_coords):
    lon_origin, lat_origin = origin
    utm_x_origin, utm_y_origin = transformer_to_utm.transform(lat_origin, lon_origin)
    
    latlon_coords = []
    for x_rel,y_rel in relative_coords:
        # Translate relative to absolute in UTM
        utm_x_coords = utm_x_origin + x_rel
        utm_y_coords = utm_y_origin + y_rel
        
        # Convert back to lon/lat
        lon,lat = transformer_to_wgs84.transform(utm_y_coords, utm_x_coords)
        latlon_coords.append((lon, lat))
    
    return latlon_coords

# Example origin point in longitude and latitude
origin = (9.286713930962872,45.623470812745815)

# Example LineString in relative coordinates (in meters)
relative_line = LineString(f1_api_coords)

# Translate relative LineString to lat/lon
relative_coords = relative_line.coords
translated_coords = relative_to_latlon(origin, relative_coords)
translated_coords = [(x/1.99, y) for x,y in translated_coords]
# Create a new LineString in longitude and latitude
translated_line = LineString(translated_coords)
list(translated_line.coords)

[(9.295865410871494, 43.04701464625807),
 (9.29589443233843, 43.047306715168226),
 (9.296072935605933, 43.04856355407001),
 (9.296123166554725, 43.048969050350244),
 (9.296376601604042, 43.050789215451296),
 (9.296467865109982, 43.051403802723954),
 (9.296807209112423, 43.0538484421562),
 (9.296875882134712, 43.0543318372904),
 (9.297062088207404, 43.0556863566111),
 (9.297071651237916, 43.05572974797384),
 (9.297238568481712, 43.05691653610353),
 (9.297272822654305, 43.05711775963511),
 (9.297537475852089, 43.05908000206144),
 (9.297658333322245, 43.059914581436125),
 (9.297794499338067, 43.06094454597623),
 (9.297957975134155, 43.06208691905801),
 (9.29796562788528, 43.06218461206249),
 (9.298143547985099, 43.06343254188928),
 (9.298295387736143, 43.06450487664922),
 (9.298468630289078, 43.065771579690235),
 (9.29851194240623, 43.06608825509181),
 (9.298773042876222, 43.06792512115053),
 (9.29883775775512, 43.068436164928045),
 (9.29895118332478, 43.069253985736324),
 (9.299132637054

In [13]:
import math 

In [14]:
centroid.x

9.286713930962872

In [15]:
# your centroid in lon,lat format  
centroid_lon, centroid_lat = (centroid.x, centroid.y)  
  
# conversion factors - these are approximations, adjust as necessary  
# 1 degree of latitude is approximately 111 km, and 1 degree of longitude is approximately 111 km multiplied by the cosine of the latitude  
km_per_degree_lat = 1 / 111  
km_per_degree_lon = 1 / (111 * math.cos(math.radians(centroid_lat)))  
  
# your array of tuples  
xy_coordinates = f1_api_coords
  
# convert each tuple in the array  
lonlat_coordinates = []  
for y,x in xy_coordinates:  
    lon = centroid_lon + (x / 10000) * km_per_degree_lon  # assuming x, y are in meters  
    lat = centroid_lat + (y / 10000) * km_per_degree_lat  # assuming x, y are in meters  
    lonlat_coordinates.append((lon,lat))  
  
print(lonlat_coordinates)  


[(9.28493498266478, 45.62276450643951), (9.284936270824302, 45.62279423616924), (9.28495301689807, 45.62292306499807), (9.284956881376631, 45.62296450643951), (9.284980068248005, 45.62315099292599), (9.28498908536465, 45.62321405598906), (9.285020001193146, 45.62346450643951), (9.28502644199075, 45.623514055989055), (9.285043188064519, 45.623652794727796), (9.28504447622404, 45.6236572992323), (9.285059934138289, 45.62377892085392), (9.28506379861685, 45.62379964157464), (9.285086985488224, 45.62400054247554), (9.28509857892391, 45.62408612806113), (9.285110172359596, 45.624191533466536), (9.285125630273845, 45.62430865058365), (9.285125630273845, 45.62431856049356), (9.285142376347613, 45.62444648842149), (9.28515654610234, 45.6245563983314), (9.285172004016589, 45.62468612806113), (9.285175868495152, 45.62471856049356), (9.285200343526045, 45.62490684878185), (9.285205496164128, 45.6249591010341), (9.285215801440293, 45.62504288481789), (9.285232547514063, 45.62517531725032), (9.2852

In [16]:
relative_line = LineString(lonlat_coordinates)

In [17]:
original_centroid

NameError: name 'original_centroid' is not defined

In [ ]:
def coordinate_shift(original_centroid, f1_api_coords):
    """This translates the original relative coordinates into longitude and latitude
    original_centroid is the centroid computed from the downloaded track data
    """

      
    # conversion factors - these are approximations, adjust as necessary  
    # 1 degree of latitude is approximately 111 km, and 1 degree of longitude is approximately 111 km multiplied by the cosine of the latitude  
    km_per_degree_lat = 1 / 111  
    km_per_degree_lon = 1 / (111 * math.cos(math.radians(centroid_lat)))  
    
    # your array of tuples  
    xy_coordinates = f1_api_coords
    
    # convert each tuple in the array  
    lonlat_coordinates = []  
    for y,x in xy_coordinates:  
        lon = centroid_lon + (x / 10000) * km_per_degree_lon  # assuming x, y are in meters  
        lat = centroid_lat + (y / 10000) * km_per_degree_lat  # assuming x, y are in meters  
        lonlat_coordinates.append((lon,lat))  
    


    relative_line = LineString(lonlat_coordinates)
    return relative_line

In [ ]:
test = coordinate_shift(centroid, f1_api_coords)

In [ ]:


def shift_centroid(original_centroid, relative_line):
    # Calculate the distance to translate in each direction  
    dx = original_centroid.x - relative_line.centroid.x  
    dy = original_centroid.y - relative_line.centroid.y  
    
    # Shift the LineString  
    shifted_line = translate(relative_line, xoff=dx, yoff=dy)  
    return shifted_line

In [ ]:
shifted_line = shift_centroid(centroid, test)

In [ ]:
# Update your GeoDataFrame  
gdf = gpd.GeoDataFrame(geometry=[shifted_line], crs="EPSG:4326")    
new_projected = gdf.to_crs(epsg=32632)  
  

kat = folium.Map(location=[centroid.y, centroid.x], zoom_start=14, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr="Esri")  
folium.GeoJson(new_projected).add_to(kat)  
kat

In [ ]:
# your centroid in lon,lat format  
centroid_lon, centroid_lat = (centroid.x, centroid.y)  
  
# conversion factors - these are approximations, adjust as necessary  
# 1 degree of latitude is approximately 111 km, and 1 degree of longitude is approximately 111 km multiplied by the cosine of the latitude  
km_per_degree_lat = 1 / 111  
km_per_degree_lon = 1 / (111 * math.cos(math.radians(centroid_lat)))  
  
# your array of tuples  
xy_coordinates = f1_api_coords
  
# convert each tuple in the array  
lonlat_coordinates = []  
for y,x in xy_coordinates:  
    lon = centroid_lon + (x / 10000) * km_per_degree_lon  # assuming x, y are in meters  
    lat = centroid_lat + (y / 10000) * km_per_degree_lat  # assuming x, y are in meters  
    lonlat_coordinates.append((lon,lat))  
  


relative_line = LineString(lonlat_coordinates)

# Your code  
gdf = gpd.GeoDataFrame(geometry=[relative_line], crs="EPSG:4326")  
new_projected = gdf.to_crs(epsg=32632)  
kat = folium.Map(location=[centroid.y, centroid.x], zoom_start=14, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr="Esri")  
folium.GeoJson(new_projected).add_to(kat)  
  
# Add a red marker  
folium.CircleMarker(  
    location=(centroid.y,centroid.x), # replace with the coordinates of the point  
    radius=5,  
    color='red',  
    fill=True,  
    fill_color='red'  
).add_to(kat)  

# Add a red marker  
folium.CircleMarker(  
    location=(relative_line.centroid.y,relative_line.centroid.x), # replace with the coordinates of the point  
    radius=5,  
    color='red',  
    fill=True,  
    fill_color='red'  
).add_to(kat)  
kat


In [ ]:
from shapely.geometry import Point  
from shapely.affinity import translate  
  
# Calculate the distance to translate in each direction  
dx = centroid.x - relative_line.centroid.x  
dy = centroid.y - relative_line.centroid.y  
  
# Shift the LineString  
shifted_line = translate(relative_line, xoff=dx, yoff=dy)  
  
# Update your GeoDataFrame  
gdf = gpd.GeoDataFrame(geometry=[shifted_line], crs="EPSG:4326")    
new_projected = gdf.to_crs(epsg=32632)  
  

kat = folium.Map(location=[centroid.y, centroid.x], zoom_start=14, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr="Esri")  
folium.GeoJson(new_projected).add_to(kat)  
kat


In [ ]:
# Your code  
gdf = gpd.GeoDataFrame(geometry=[relative_line], crs="EPSG:4326")  
new_projected = gdf.to_crs(epsg=32632)  
kat = folium.Map(location=[centroid.y, centroid.x], zoom_start=14, tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr="Esri")  
folium.GeoJson(new_projected).add_to(kat)  
  
# Add a red marker  
folium.CircleMarker(  
    location=(centroid.y,centroid.x), # replace with the coordinates of the point  
    radius=5,  
    color='red',  
    fill=True,  
    fill_color='red'  
).add_to(kat)  

# Add a red marker  
folium.CircleMarker(  
    location=(relative_line.centroid.y,relative_line.centroid.x), # replace with the coordinates of the point  
    radius=5,  
    color='red',  
    fill=True,  
    fill_color='red'  
).add_to(kat)  
kat


In [ ]:
centroid.x

9.286713930962872

In [ ]:
centroid.y

45.623470812745815

In [ ]:
45.623470812745815  9.286713930962872


In [ ]:
lap_55_1

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z
3,2023-09-02 14:03:48.251,0 days 00:18:48.140000,,194.836111,0 days 00:00:00.074000,6938,80,2,19,False,8,car,1.644444,0.000300,OnTrack,-925.026682,2361.235516,-1.369417e-43
5,2023-09-02 14:03:48.571,0 days 00:18:48.460000,16,194.836111,0 days 00:00:00.394000,6846,80,2,19,False,8,car,8.755556,0.001598,OnTrack,-922.578445,2432.253942,4.320352e-43
7,2023-09-02 14:03:48.811,0 days 00:18:48.700000,16,201.769444,0 days 00:00:00.634000,6975,80,2,22,False,8,car,14.088889,0.002571,OnTrack,-921.059064,2490.247295,-7.593702e-43
10,2023-09-02 14:03:49.171,0 days 00:18:49.060000,16,212.669444,0 days 00:00:00.994000,7050,82,2,22,False,8,car,22.288889,0.004068,OnTrack,-921.001934,2560.970811,-2.497971e-41
12,2023-09-02 14:03:49.331,0 days 00:18:49.220000,16,217.647222,0 days 00:00:01.154000,7075,82,2,21,False,8,car,25.933333,0.004733,OnTrack,-920.997597,2596.464659,4.395010e-40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,2023-09-02 14:05:30.731,0 days 00:20:30.620000,16,607.788333,0 days 00:01:42.554000,10995,306,8,100,False,8,car,5367.490278,0.979669,OnTrack,-1450.887667,-1706.074547,1.867094e+03
754,2023-09-02 14:05:31.011,0 days 00:20:30.900000,16,610.121667,0 days 00:01:42.834000,11034,310,8,100,False,8,car,5391.601389,0.984070,OnTrack,-1434.361278,-1464.244576,1.868827e+03
756,2023-09-02 14:05:31.251,0 days 00:20:31.140000,16,612.188333,0 days 00:01:43.074000,11120,312,8,100,False,8,car,5412.401389,0.987867,OnTrack,-1421.531174,-1286.997963,1.868830e+03
759,2023-09-02 14:05:31.491,0 days 00:20:31.380000,16,614.121667,0 days 00:01:43.314000,11171,313,8,100,False,8,car,5433.268056,0.991675,OnTrack,-1406.935746,-1088.129165,1.870007e+03


In [ ]:
plt.figure(figsize=(8,8))  
  
# Get the current axes, creating one if necessary.  
ax = plt.gca()  
  
# Set the facecolor of the axes:  
ax.set_facecolor('white')  
  
plt.plot(lap_55_1['X'], lap_55_1['Y'], label='Driver 1')  
#plt.plot(lap_55_2['X'], lap_55_2['Y'], label='Driver 55')  
  
plt.plot(0, 0, marker='o', markersize=5, label='Point (0,0)', color='blue') # Add a blue dot at (0,0)  
  
plt.xlabel('Relative X coordinate in decimetre compared to origon')  
plt.ylabel('Y')  
plt.ylim(-6000,17000)  
plt.xlim(-6000,17000)  
  
plt.title('Monza track')  
#plt.legend()  # Show legend with labels  
  
plt.tight_layout()  # Adjust the layout  
plt.savefig('figure.png')  # Save the figure  
plt.show()  



NameError: name 'plt' is not defined

In [ ]:
plt.plot(lap_55_1['X'][80:90], lap_55_1['Y'][80:90], label='Driver 55 Lap 1')
plt.plot(lap_55_2['X'][45:55], lap_55_2['Y'][45:55], label='Driver 55 Lap 2')

plt.xlabel('X')
plt.ylabel('Y')
plt.title('Trajectory Comparison')
plt.legend()  # Show legend with labels

plt.show()